In [1]:
from __future__ import print_function 


import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline

PATH = 'data/mnist.npz'

def load_data(path=PATH):
    if os.path.exists(path):
        data = np.load(path)
        return data
    else:
        return tf.keras.datasets.mnist.load_data()

In [2]:
DATA = load_data()

def reshape_data(data = DATA):
    X_train, y_train = data['x_train'], data['y_train']
    X_test, y_test = data['x_test'], data['y_test']
    
    X_train, X_test = X_train.astype(np.float32).reshape(-1, 28*28)/255.0,X_test.astype(np.float32).reshape(-1, 28*28)/255.0
    y_train, y_test = y_train.astype(np.int32), y_test.astype(np.int32)
    
    return X_train, y_train, X_test, y_test

In [32]:
X_train, y_train, X_test, y_test = reshape_data()

In [39]:
#parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

#network parameters 
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [40]:
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [41]:
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.add(tf.matmul(layer_2, weights['out']) ,biases['out'])
    return out_layer

In [42]:
logits = neural_net(X)

loss_optimizer = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_optimizer = optimizer.minimize(loss_optimizer)

correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [49]:
#training
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = tf.train.batch([X_train, y_train],
                    batch_size=batch_size, allow_smaller_final_batch=True)
        
        sess.run(training_optimizer, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            loss, train_acc = sess.run([loss_optimizer, accuracy], 
                                       feed_dict={X:batch_x, Y:batch_y})
            print("Step: " + str(step) + ", Minibatch Loss= " +\
                 "{:.4f}".format(loss) + ", Training Accuracy= " +\
                 "{:.3f}".format(train_acc))
            
    print("optimization Finished!")
    
    print("Testing accuracy: ", \
            sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("batch_5:0", shape=(?, 60000, 784), dtype=float32) which was passed to the feed with key Tensor("Placeholder_2:0", shape=(?, 784), dtype=float32).